<a href="https://colab.research.google.com/github/hyetae/2023-graduate/blob/main/crop/image_crop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setting

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
import matplotlib.pyplot as plt

#Logic...
##1. 핵 찾기(이때 size 조절)
##2. 핵 주변의 색 분포 파악
##3. 세포가 한 개만 있으면 세포질을 찾아 bounding box 그리기
##4. 세포가 여러 개 있으면 핵 주변만 morphology 진행
##5. 2번으로 돌아가 반복

In [81]:
def find_contours(img, size):
  RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  gray = cv2.cvtColor(RGB, cv2.COLOR_RGB2GRAY)
  res, thr = cv2.threshold(gray, size, 255, cv2.THRESH_BINARY)
  contours, _ = cv2.findContours(thr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  
  ct = []
  for i in range(len(contours)-1):
    cnt = contours[i+1]
    x, y, w, h = cv2.boundingRect(cnt)
    if w+h < 80:
      continue
    else:
      ct.append(cnt)
  
  #cv2_imshow(thr)
  return ct

In [82]:
def color_distribution(img):
  colors = ['b', 'g', 'r']
  bgr_planes = cv2.split(img)
  global blue

  for (p, c) in zip(bgr_planes, colors):
    if c == 'b':
      blue = cv2.calcHist([p], [0], None, [256], [0, 256])
    hist = cv2.calcHist([p], [0], None, [256], [0, 256])

  tmp1 = blue[195].tolist()
  tmp2 = int(tmp1[0])
  return tmp2

In [83]:
def draw_bb(img, contours):
  for i in range(len(contours)):
    cnt = contours[i]
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 3)

  cv2_imshow(img)

In [84]:
def erode(img):
  try:
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    erosion = cv2.erode(img,kernel,iterations=2)
  except:
    return None
  return erosion

In [85]:
def dilate(img):
  try:
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dilation = cv2.dilate(img,kernel,iterations=3)
  except:
    return None
  return dilation

In [91]:
def find_cells(img):

  img_copy = img.copy()

  isTrue = True

  #핵 주변 색 분포 파악
  while(isTrue):

    #핵 찾기
    contours = find_contours(img_copy, 110)

    #핵 주변 찾기
    for i in range(len(contours)):
      cnt = contours[i]
      x, y, w, h = cv2.boundingRect(cnt)
      img_cut = img_copy[y-h:y+2*h, x-w:x+2*w]

      #색 분포 정상
      if color_distribution(img_cut) < 300:
          isTrue = False

      #색 분포 비정상
      else:
        isTrue = True
        morph = erode(dilate(img_cut))
        if morph is None:
          return None
        else:
          img_copy[y-h:y+2*h, x-w:x+2*w] = morph

  #세포질 찾기
  return find_contours(img_copy, 190)

  #바운딩 박스 그리기
  #draw_bb(img, contours)

In [87]:
def crop_imgs(img, contours, path, name):
  img_copy = img.copy()
  for i in range(len(contours)):
      if i == 0:
        continue
      cnt = contours[i]
      x, y, w, h = cv2.boundingRect(cnt)
      img_cut = img_copy[y:y+h, x:x+w]
      f = path + name[:-4] + '-' + str(i) + '.jpg'
      cv2.imwrite(f, img_cut)
      #cv2_imshow(img_cut)

  print(f + ' done')

In [88]:
def rename(path):
  for i in path:
    path_list = os.listdir(i)
    jpg_list = [file for file in path_list if file.endswith('.jpg')]
    
    for j in jpg_list:
      uni = unicodedata.normalize('NFC',j)
      if uni[:4] == '미세먼지':
        uni = uni.replace('미세먼지', 'fine-dust') 
        os.rename(i+j, i+uni)
        print(i+uni)
      elif uni[:5] == '초미세먼지':
        uni = uni.replace('초미세먼지', 'ultra-fine-dust') 
        os.rename(i+j, i+uni)
        print(i+uni)

def no_space(path):
  for i in path:
    path_list = os.listdir(i)
    jpg_list = [file for file in path_list if file.endswith('.jpg')]
    
    for j in jpg_list:
      re = j.replace(' ', '') 
      os.rename(i+j, i+re)

In [92]:
import os
import unicodedata

def main():
  input = ['/content/drive/MyDrive/graduate/raw_cell_images/BALFDiff-Quik/',
          '/content/drive/MyDrive/graduate/raw_cell_images/BALFDiff-Quik-2/',
          '/content/drive/MyDrive/graduate/raw_cell_images/Diff(PM0.1)/2020-11-23BALdiffControl/',
          '/content/drive/MyDrive/graduate/raw_cell_images/Diff(PM0.1)/2020-11-24BALdiffGroupI/',
          '/content/drive/MyDrive/graduate/raw_cell_images/Diff(PM0.1)/2020-11-25BALdiffGroupII/',
          '/content/drive/MyDrive/graduate/raw_cell_images/Diff(PM0.1)/2020-11-26BALdiffGroupIII/',
          '/content/drive/MyDrive/graduate/raw_cell_images/Diff(PM0.1)/2020-11-27BALdiffGroupIV/']
  
  output = ['/content/drive/MyDrive/graduate/cell_crop_images/BALFDiff-Quick/',
          '/content/drive/MyDrive/graduate/cell_crop_images/BALFDiff-Quick-2/',
          '/content/drive/MyDrive/graduate/cell_crop_images/Diff(PM0.1)/2020-11-23BALdiffControl/',
          '/content/drive/MyDrive/graduate/cell_crop_images/Diff(PM0.1)/2020-11-24BALdiffGroupI/',
          '/content/drive/MyDrive/graduate/cell_crop_images/Diff(PM0.1)/2020-11-24BALdiffGroupII/',
          '/content/drive/MyDrive/graduate/cell_crop_images/Diff(PM0.1)/2020-11-24BALdiffGroupIII/',
          '/content/drive/MyDrive/graduate/cell_crop_images/Diff(PM0.1)/2020-11-27BALdiffGroupIV/']

  #rename(input)
  #no_space(input)

  for i,o in zip(input, output):
    input_list = os.listdir(i)
    jpg_list = [file for file in input_list if file.endswith('.jpg')]
    for j in jpg_list:
      if j[-6:] == '.5.jpg':
        continue
      f = i + j
      img = cv2.imread(f)
      
      if img is None:
        print(f + ' pass')
        continue

      print(f + ' start')
      contours = find_cells(img)

      if contours is None:
        continue
      else:
        crop_imgs(img, contours, o, j)


In [ ]:
if __name__ == "__main__":
	main()

/content/drive/MyDrive/graduate/raw_cell_images/BALFDiff-Quik/fine-dustGroupXI#10x200-3.jpg start
/content/drive/MyDrive/graduate/cell_crop_images/BALFDiff-Quick/fine-dustGroupXI#10x200-3-213.jpg done
/content/drive/MyDrive/graduate/raw_cell_images/BALFDiff-Quik/fine-dustGroupIX#2x200-4.jpg start
/content/drive/MyDrive/graduate/raw_cell_images/BALFDiff-Quik/fine-dustGroupXII#2x200-2.jpg start
/content/drive/MyDrive/graduate/raw_cell_images/BALFDiff-Quik/fine-dustGroupX#1x200-2.jpg start
/content/drive/MyDrive/graduate/raw_cell_images/BALFDiff-Quik/fine-dustGroupXII#4x200-4.jpg start
/content/drive/MyDrive/graduate/raw_cell_images/BALFDiff-Quik/fine-dustGroupIX#9x200-1.jpg start
/content/drive/MyDrive/graduate/raw_cell_images/BALFDiff-Quik/fine-dustGroupXII#4x200-5.jpg start
/content/drive/MyDrive/graduate/raw_cell_images/BALFDiff-Quik/fine-dustGroupIX#4x200-3.jpg start
/content/drive/MyDrive/graduate/cell_crop_images/BALFDiff-Quick/fine-dustGroupIX#4x200-3-77.jpg done
/content/drive/My